In [ ]:
from flask import Flask, render_template, request, g, session, redirect, url_for
from flask_cors import CORS, cross_origin
import json
import db_access # connection info for all dbs
import my_postgres as pg # our postgres methods
import my_redis as rd # our redis methods
import psycopg2 # lib to connect to postgres
import redis # lib to connect to redis

#database connections
pgconn = psycopg2.connect(**db_access.postgres)
rdconn = redis.Redis(**db_access.redis)

# general flask setup
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'
app.secret_key = "top secret key of awesomeness"

# front page, returns user session
@app.route('/')
def main():
    if "user" in session:
        return json.dumps(session["user"], ensure_ascii=False)
    else:
        return "YOU ARE NOT A USER!!!!!"

    
    
############################
#     POSTGRES / USERS     #
############################
    
#login, can test with below endpoint
#http://127.0.0.1:5000/login/nancy1@claudia.io/1234
@app.route('/login/<email>/<password>')
def set_session(email, password):
    user = pg.login(pgconn, email, password)
    if user is None:
        return "Wrong email or password"
    else:
        session["user"] = user
        return redirect(url_for("main"))

#logout
@app.route('/logout')
def logout():
    session.pop("user", None)
    return redirect(url_for("main"))

# get user from email
@app.route('/user/<email>')
def get_user_info(email):
    return json.dumps(pg.get_user_info(pgconn, email), ensure_ascii=False)

# create user
# {
#     "name": "daniel",
#     "email": "da@ni.el",
#     "phone": "12345678",
#     "street": "vej24",
#     "password": "1234",
#     "zip": "2680"
# }
@app.route('/user/create', methods=["POST"])
def create_user():
    user = request.json
    return json.dumps(pg.create_user(pgconn, user), ensure_ascii=False)

# delete user from email
@app.route('/user/delete/<email>')
def delete_user(email):
    user = request.json
    pg.delete_user(pgconn, email)
    return json.dumps({'success':True}), 200


#############################
#       REDIS / CART        #
#############################
@app.route('/cart/get')
def get_cart():
    if "user" not in session:
        return json.dumps({})
    
    return json.dumps(rd.get_cart(rdconn, session["user"]["email"]))

@app.route('/cart/add', methods=["POST"])
def add_to_cart():
    item = request.json.get('item')
    count = request.json.get('count')
    if "user" not in session or item is None or count is None:
        return json.dumps({'success':False}), 400
    
    json.dumps(rd.add_to_cart(rdconn, session["user"]["email"], int(item), int(count)))
    return json.dumps({'success':True}), 200


@app.route('/cart/remove', methods=["POST"])
def remove_from_cart():
    item = request.json
    if "user" in session and type(item) is int:
        rd.remove_from_cart(rdconn, session["user"]["email"], item)
        
    return json.dumps({'success':True}), 200


@app.route('/cart/clear')
def clear_cart():
    if "user" in session:
        rd.clear_cart(rdconn, session["user"]["email"])
        
    return json.dumps({'success':True}), 200

# start app
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/May/2020 17:01:57] "GET /user/da@ni.el HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2020 17:02:01] "GET /user/delete/da@ni.el HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2020 17:02:03] "GET /user/delete/da@ni.el HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2020 17:02:06] "GET /user/da@ni.el HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2020 17:02:07] "GET /user/da@ni.el HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2020 17:02:08] "POST /user/create HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2020 17:02:09] "POST /user/create HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2020 17:02:11] "GET /user/da@ni.el HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2020 17:02:12] "GET /user/da@ni.el HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2020 17:02:16] "GET /user/delete/da@ni.el HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2020 17:02:17] "POST /user/create HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2020 17:02:19] "GET /user/da@ni.el HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2020 17:02:25] "GET /use